In [2]:
%load_ext autotime

In [3]:
import nltk
import pickle
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tools.functions

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# nltk.download('stopwords')
# nltk.download('wordnet')

time: 2.23 s


In [4]:
vectorizer_max_features = 1500

time: 235 µs


In [5]:
all_text = pd.read_csv('archive/sub_reddits.csv')
all_text['selftext'] = all_text['selftext'].astype(str)

time: 6.5 s


In [6]:
sanitized_posts_df = pd.DataFrame(tools.functions.sanitize_posts(all_text['selftext']), columns=['posts'])
sanitized_posts_df = sanitized_posts_df[sanitized_posts_df['posts'] != '']
sanitized_posts_df = sanitized_posts_df[sanitized_posts_df['posts'] != 'a']
sanitized_posts_df = sanitized_posts_df[sanitized_posts_df['posts'] != '_']
sanitized_posts_df = sanitized_posts_df[sanitized_posts_df['posts'].str.contains('[^\d^\s]')]

time: 2min 13s


In [20]:
# Workspace for additional text cleaning

time: 703 µs


# The following section runs a Latent Semantic Analysis (LSA) on the corpus using TruncatedSVD

In [21]:
count_vectorizer = CountVectorizer(min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
count_vectorized_posts = count_vectorizer.fit_transform(sanitized_posts_df['posts'])



time: 21.3 s


In [26]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=2, n_iter=7, random_state=42)
svd_fit_transform = svd.fit_transform(count_vectorized_posts)


print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())
print(svd.singular_values_)

[0.13418152 0.11829433]
0.25247584468078527
[3170.21868301 2869.74127917]
time: 5 s


In [23]:
topic_word = pd.DataFrame(svd.components_.round(3), index=["component_1", "component_2"], columns=vectorizer.get_feature_names())
topic_word

,00,000,0000,00000,000000,000001,00001,0001,0001104659,0001193125,...,то,уоu,что,это,ісо,الاستثمار,في,كما,من,ﬁnancial
component_1,0.037,0.024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
component_2,-0.012,-0.005,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0


time: 63.8 ms


In [24]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

time: 478 µs


In [25]:
display_topics(svd, vectorizer.get_feature_names(), 5)


Topic  0
amp, http, com, gt, earnings

Topic  1
comment, wallstreetbets, 17, 15, spy
time: 37.7 ms


In [28]:
Vt = pd.DataFrame(svd_fit_transform.round(5),
             index = sanitized_posts_df['posts'],
             columns = ["component_1","component_2" ])
Vt

,component_1,component_2
posts,,
so m sober a judge about to go to sleep when have this idea and get up and go write it down in notepad because that a good idea then get better one which is to submit it here so here is what wrote expected variance over time option put and call price are they linear function with respect to time maximize call and putt cost v gross rev combined with prob dist of stock price profit maybe if we get something going we can submit it to programing and get working program use subreddits like department lol,1.14076,-0.32376
i just sold my first house project house that ve lived in while renovating part time for the past 3 year and have 15 000 in profit that need to invest the money will be used a part of down payment on my next house in roughly 2 5 year have couple of idea but would like to hear what you would do in this situation edit what about etf a large portion of this investment pro con personal thought favorite fund etc,0.88391,-0.18157
we ve all heard the story about those numerous non penny stock that sold at penny level on thursday to me it pretty obvious what happened the s government must have instructed some of the largest volume trader institutional trader to dump large volume of those stock at penny level right at 2 38pm eastern time on thursday in an orchestrated effort to drive down key s stock market index and thereby dramatically devalue the s stock market objection 1 wouldn those institution lose combined billion of dollar by dumping those stock at penny level yes indeed they would but believe that the s federal reserve promised to recoup those institution whatever loss they incurred objection 2 it no secret that china among other country ha been trying to slow down it economy to stop t currency from being too quickly devalued is it so hard to imagine then that the s would want to do the same thing only more covertly the s dollar ha been severely devalued recently until this week that is one ha to wonder which firm gained from all of of these s stock being sold off at penny level suspect they could have been chinese investor in other word it very possible that china just took gigantic windfall profit at the expense of the s stock market precisely when they didn want to take one did the s market manipulation capability just assert it dominance over chinese market manipulation capability know it seems counter intuitive but so doe the idea of slowing down your own economy in order to bolster your currency would not be at all surprised to see this currency warfare continue for month until the big chinese firm learn how to fight back thursday just proved that american innovation is leap and bound ahead of the chinese in regard to market and currency manipulation the dollar is ridiculously stronger today than it wa only few day ago against both the euro and the yuan,4.34081,-1.40833
i want to learn about all type of investing then may decide to focus and study on certain area more specifically will shortly have lot more time on my hand and love to read and study new thing doe anyone have any resource they can share be it book to buy online literature etc,0.42224,-0.12648
can someone recommend good charting site where can see all of my stock in big long list with each displaying one year chart option for 2 3 5 would be nice preferably with moving average rsi etc a it is now use yahoo finance and clicking through to each stock 30 just take way too long,1.09618,-0.32689
...,...,...
disclaimer am long equity please do due diligence this is based off of 60 minute of quick analysis company overview xeris is spec pharma company founded in 2005 their primary scope of work involves developed injectable and infusible drug 2019 present product launch their first product wa approved in september 2019 called gvoke it is pfs and auto injector that ha glucagon to treat severe hypoglycemia this is market a two different product gvoke pfs nov 2019 and gvoke hypopen july 2020 competition primary competitio

time: 25 ms


# TODO Figure out cosine_similarity

In [33]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity((topic_word.iloc[0], topic_word.iloc[1])).round()

array([[1., 0.],
       [0., 1.]])

time: 4.09 ms


# Implementing NMF

In [7]:
NMF_vectorizer = CountVectorizer(stop_words='english')
NMF_posts = NMF_vectorizer.fit_transform(sanitized_posts_df['posts'])

time: 20.3 s


In [37]:
from sklearn.decomposition import NMF

nmf_model = NMF(2, random_state=42)
NMF_nmf = nmf_model.fit_transform(NMF_posts)

time: 8.25 s


In [38]:
NMF_df = pd.DataFrame(nmf_model.components_.round(3), index=['component_1', 'component_2'], columns=NMF_vectorizer.get_feature_names())

time: 382 ms


In [39]:
NMF_df

,00,000,0000,00000,000000,00000000,000000000,000000000000000001,00000000000000f34b57e3bee97172558574a2b2a5d50e20e23b45e28955673f,0000000000000558,...,𝗽𝗿𝗶𝗰𝗲,𝘁𝗵𝗮𝗻,𝘁𝗵𝗲,𝘐𝘯𝘵𝘦𝘭𝘭𝘪𝘨𝘦𝘯𝘵,𝘐𝘯𝘷𝘦𝘴𝘵𝘰𝘳,𝘛𝘩𝘦,𝚃𝚊𝚗𝚔,𝟐𝟓,𝟙𝟟,𝟚𝟙𝟝ℙ
component_1,2.284,1.426,0.003,0.0,0.002,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
component_2,0.000,0.105,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


time: 16.8 ms


In [41]:
display_topics(nmf_model, NMF_vectorizer.get_feature_names(), 10)


Topic  0
amp, http, gt, com, earnings, stock, market, ha, click, company

Topic  1
comment, wallstreetbets, 17, 15, spy, 24, 19, 20, www, reddit
time: 295 ms


# Latent Dirichlet Allocation

In [9]:
LDA_vectorizer = CountVectorizer(strip_accents='unicode', stop_words='english', lowercase=True, token_pattern=r'\b[a-zA-Z]{3,}\b', max_df=0.5, min_df=10)

LDA_transformed = LDA_vectorizer.fit_transform(sanitized_posts_df['posts'])

time: 17 s


In [10]:
LDA_Tf_vectorizer = TfidfVectorizer(**LDA_vectorizer.get_params())

tfidfvectorized_posts = LDA_Tf_vectorizer.fit_transform(sanitized_posts_df['posts'])

time: 17.6 s


In [11]:
tfidf_df = pd.DataFrame(tfidfvectorized_posts.toarray(), columns=LDA_vectorizer.get_feature_names())
tfidf_df.head()

,aaa,aaaand,aaba,aaii,aal,aamrq,aaoi,aap,aapl,aapls,...,zuck,zuckerberg,zuckerman,zumiez,zumz,zuo,zuora,zurich,zweig,zynga
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


time: 17.3 s


In [12]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf.fit(LDA_transformed)

# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(tfidfvectorized_posts)

LatentDirichletAllocation(n_components=20, random_state=0)

time: 39min 22s


In [13]:
with open('archive/lda.20.0.pkl', 'wb') as picklefile:
    pickle.dump(lda_tfidf, picklefile)

time: 6.46 ms


In [ ]:
with open('archive/lda.20.0.pkl', 'rb') as picklefile:
    lda_tfidf = pickle.load(picklefile)

In [15]:
trans_lda = lda_tfidf.transform(tfidfvectorized_posts)

time: 56.1 s


In [16]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda_tf, LDA_transformed, LDA_Tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
16     0.170854  0.081105       1        1  10.051957
14     0.096329  0.010317       2        1   9.672610
0      0.117943  0.011978       3        1   7.939382
17     0.081844 -0.059331       4        1   6.272591
13     0.151227 -0.014288       5        1   6.251314
19     0.039243 -0.097801       6        1   6.208835
18    -0.048781 -0.134901       7        1   5.468649
8      0.041338 -0.113085       8        1   5.302143
2     -0.069840 -0.089253       9        1   4.997633
15     0.071368 -0.031610      10        1   4.973416
7     -0.335464  0.053075      11        1   4.775851
12    -0.057282 -0.126025      12        1   4.548217
4      0.138941  0.088672      13        1   4.435691
10     0.153499  0.010532      14        1   4.075174
11     0.068497 -0.035052      15        1   3.808120
3     -0.174072 -0.098522      16        1   3.495586
5      0.089671  0.046757      17        1   2.608732
1     -0.198605 -0.033896      18        1   1.912343
9     -0.037842  0.453900      19        1   1.905053
6     -0.298866  0.077428      20        1   1.296703, topic_info=          Term           Freq          Total Category  logprob  loglift
10808     http  162320.000000  162320.000000  Default  30.0000  30.0000
4302       com  140103.000000  140103.000000  Default  29.0000  29.0000
851        amp  128712.000000  128712.000000  Default  28.0000  28.0000
25456      www   79595.000000   79595.000000  Default  27.0000  27.0000
9251      fund   58015.000000   58015.000000  Default  26.0000  26.0000
...        ...            ...            ...      ...      ...      ...
1113     apple    1781.822175   11228.804813  Topic20  -4.6866   2.5045
22908    title     930.364826    4185.598766  Topic20  -5.3364   2.8415
19015  revenue    1129.776399   19795.581485  Topic20  -5.1422   1.4819
261     actual     669.529443    5372.446457  Topic20  -5.6654   2.2629
13923  mention     654.437755    4164.796220  Topic20  -5.6882   2.4947

[1733 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
5          5  0.973416       aamrq
8          2  0.007215        aapl
8          3  0.233549        aapl
8          5  0.024664        aapl
8          6  0.009060        aapl
...      ...       ...         ...
25693     20  0.002175      zillow
25727      9  0.979546  zuckerberg
25727     11  0.012044  zuckerberg
25727     12  0.004015  zuckerberg
25734     10  0.974659       zweig

[9895 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 15, 1, 18, 14, 20, 19, 9, 3, 16, 8, 13, 5, 11, 12, 4, 6, 2, 10, 7])

time: 1min 39s


In [17]:
pyLDAvis.sklearn.prepare(lda_tfidf, tfidfvectorized_posts, LDA_Tf_vectorizer)

/Users/albertlee/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.088984 -0.010103       1        1  9.732127
16     0.066517 -0.116409       2        1  8.610916
14     0.058695  0.063791       3        1  8.084206
19     0.076325  0.050240       4        1  7.793552
18     0.048399 -0.003160       5        1  7.352967
13     0.066942 -0.097485       6        1  6.517987
10     0.048854 -0.113551       7        1  5.222313
4      0.050081 -0.165103       8        1  5.122373
2      0.039130 -0.041090       9        1  5.069122
8      0.049646  0.077742      10        1  4.573072
6      0.057618  0.115947      11        1  4.548502
5      0.056517 -0.126331      12        1  4.382179
12     0.035320  0.064342      13        1  4.301622
7     -0.032515  0.105898      14        1  4.035689
15     0.030196 -0.019340      15        1  3.332033
11     0.054425 -0.039027      16        1  3.032190
17     0.012691  0.185959      17        1  2.764917
3     -0.022553  0.196506      18        1  2.257755
9     -0.319583 -0.234607      19        1  2.064622
1     -0.465688  0.105779      20        1  1.201854, topic_info=          Term          Freq         Total Category  logprob  loglift
14731      nan  12447.000000  12447.000000  Default  30.0000  30.0000
10808     http   8406.000000   8406.000000  Default  29.0000  29.0000
4302       com   7526.000000   7526.000000  Default  28.0000  28.0000
851        amp   6081.000000   6081.000000  Default  27.0000  27.0000
25456      www   4915.000000   4915.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
8318      fcel     15.895119     22.200160  Topic20  -7.0435   4.0872
22943    toast     17.745408     27.946276  Topic20  -6.9333   3.9671
23101  trading     78.437739   3393.563818  Topic20  -5.4472   0.6540
6708    doomed     17.793796     58.712633  Topic20  -6.9306   3.2275
14823      nbg     17.746893     65.857046  Topic20  -6.9333   3.1100

[1686 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
50        10  0.935959     abn
54         3  0.018420  aboard
54         4  0.036840  aboard
54        10  0.055260  aboard
54        11  0.865735  aboard
...      ...       ...     ...
25651     10  0.029050    yuan
25651     13  0.014525    yuan
25651     14  0.043576    yuan
25651     18  0.014525    yuan
25681     14  0.937761   zesty

[10451 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 17, 15, 20, 19, 14, 11, 5, 3, 9, 7, 6, 13, 8, 16, 12, 18, 4, 10, 2])

time: 54.9 s


In [18]:
pyLDAvis.sklearn.prepare(lda_tf, LDA_transformed, LDA_Tf_vectorizer, mds='mmds')

/Users/albertlee/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
16     0.191477  0.131114       1        1  10.051957
14     0.227582  0.048283       2        1   9.672610
0      0.015492  0.259111       3        1   7.939382
17    -0.167497  0.218824       4        1   6.272591
13     0.045758  0.140134       5        1   6.251314
19    -0.010128 -0.019325       6        1   6.208835
18    -0.079642 -0.166128       7        1   5.468649
8     -0.170718  0.091573       8        1   5.302143
2      0.138562 -0.200146       9        1   4.997633
15     0.124411 -0.056772      10        1   4.973416
7      0.152097 -0.455616      11        1   4.775851
12    -0.283851  0.113143      12        1   4.548217
4      0.240981  0.242423      13        1   4.435691
10     0.103407  0.258155      14        1   4.075174
11    -0.075072  0.298092      15        1   3.808120
3     -0.352516 -0.047167      16        1   3.495586
5     -0.147664 -0.048566      17        1   2.608732
1     -0.316509 -0.228185      18        1   1.912343
9      0.519429 -0.109147      19        1   1.905053
6     -0.155600 -0.469798      20        1   1.296703, topic_info=          Term           Freq          Total Category  logprob  loglift
10808     http  162320.000000  162320.000000  Default  30.0000  30.0000
4302       com  140103.000000  140103.000000  Default  29.0000  29.0000
851        amp  128712.000000  128712.000000  Default  28.0000  28.0000
25456      www   79595.000000   79595.000000  Default  27.0000  27.0000
9251      fund   58015.000000   58015.000000  Default  26.0000  26.0000
...        ...            ...            ...      ...      ...      ...
1113     apple    1781.822175   11228.804813  Topic20  -4.6866   2.5045
22908    title     930.364826    4185.598766  Topic20  -5.3364   2.8415
19015  revenue    1129.776399   19795.581485  Topic20  -5.1422   1.4819
261     actual     669.529443    5372.446457  Topic20  -5.6654   2.2629
13923  mention     654.437755    4164.796220  Topic20  -5.6882   2.4947

[1733 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
5          5  0.973416       aamrq
8          2  0.007215        aapl
8          3  0.233549        aapl
8          5  0.024664        aapl
8          6  0.009060        aapl
...      ...       ...         ...
25693     20  0.002175      zillow
25727      9  0.979546  zuckerberg
25727     11  0.012044  zuckerberg
25727     12  0.004015  zuckerberg
25734     10  0.974659       zweig

[9895 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 15, 1, 18, 14, 20, 19, 9, 3, 16, 8, 13, 5, 11, 12, 4, 6, 2, 10, 7])

time: 1min 40s


In [19]:
pyLDAvis.sklearn.prepare(lda_tf, LDA_transformed, LDA_Tf_vectorizer, mds='tsne')

/Users/albertlee/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
16     -10.878068   10.538728       1        1  10.051957
14     -60.884026  -75.591820       2        1   9.672610
0       -2.336119  -53.289074       3        1   7.939382
17     -61.385319  -12.914634       4        1   6.272591
13     -28.737743  126.296997       5        1   6.251314
19       1.066606 -123.552811       6        1   6.208835
18      45.140720   -7.947869       7        1   5.468649
8     -140.795975  -98.530548       8        1   5.302143
2       61.531681  -83.240562       9        1   4.997633
15     -72.563057 -140.396881      10        1   4.973416
7       15.984491   63.578186      11        1   4.775851
12    -160.851624   68.945145      12        1   4.548217
4       85.961830   51.922935      13        1   4.435691
10     -99.020836  109.936348      14        1   4.075174
11    -176.764130   -8.681684      15        1   3.808120
3     -117.189880  -37.420200      16        1   3.495586
5      108.829918  -22.946203      17        1   2.608732
1       47.588097  121.934814      18        1   1.912343
9     -106.467834   30.176889      19        1   1.905053
6      -50.613804   59.328842      20        1   1.296703, topic_info=          Term           Freq          Total Category  logprob  loglift
10808     http  162320.000000  162320.000000  Default  30.0000  30.0000
4302       com  140103.000000  140103.000000  Default  29.0000  29.0000
851        amp  128712.000000  128712.000000  Default  28.0000  28.0000
25456      www   79595.000000   79595.000000  Default  27.0000  27.0000
9251      fund   58015.000000   58015.000000  Default  26.0000  26.0000
...        ...            ...            ...      ...      ...      ...
1113     apple    1781.822175   11228.804813  Topic20  -4.6866   2.5045
22908    title     930.364826    4185.598766  Topic20  -5.3364   2.8415
19015  revenue    1129.776399   19795.581485  Topic20  -5.1422   1.4819
261     actual     669.529443    5372.446457  Topic20  -5.6654   2.2629
13923  mention     654.437755    4164.796220  Topic20  -5.6882   2.4947

[1733 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
5          5  0.973416       aamrq
8          2  0.007215        aapl
8          3  0.233549        aapl
8          5  0.024664        aapl
8          6  0.009060        aapl
...      ...       ...         ...
25693     20  0.002175      zillow
25727      9  0.979546  zuckerberg
25727     11  0.012044  zuckerberg
25727     12  0.004015  zuckerberg
25734     10  0.974659       zweig

[9895 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 15, 1, 18, 14, 20, 19, 9, 3, 16, 8, 13, 5, 11, 12, 4, 6, 2, 10, 7])

time: 1min 34s
